In [3]:
import pandas as pd
import json
import random
from sklearn.model_selection import train_test_split

# from transformers import  AutoTokenizer
model_name = "/home/odedh/SML-For-Debug/models/Qwen3-0.6B"

SEED = 42
random.seed(SEED)

In [4]:
# Load the dataset from the Excel file
file_path = "/home/odedh/SML-For-Debug/data/dataset/PyresBugs.xlsx"  # Replace with the actual file path
df = pd.read_excel(file_path)

# Display the first few rows of the dataset
df.head(-10)

,Bug_Description,Bug_Type,CVE-ID,Commit_URL,Commit_sha,Dataset_input,Diff_patch,Fault Free Code,Faulty Code,Fixed_Method,Impact,Implementation-Level Description,Contextual-Level Description,High-Level Description,Project,Python_Version,Test_File_Path,Url,Fault_Acronym
0,Merge pull request from GHSA-p867-fxfr-ph2w\n\...,security,CVE-2022-23651,https://github.com/python/cpython/commit/62476...,62476638986e5b6d7459aca5ef8ce220760226e0,CVEFixes,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...,"def _make_sqlite_account_info(self, env=None, ...","def _make_sqlite_account_info(self, env=None, ...","_make_sqlite_account_info(self, env=None, last...",NaN,Alter the behavior of the _make_sqlite_account...,Modify the _make_sqlite_account_info method to...,Modify the _make_sqlite_account_info method to...,b2-sdk,3.10.0,test_account_info.py,https://github.com/Backblaze/b2-sdk-python,WPFV
1,Prevent setting swift+config locations\n\nForb...,security,NaN,https://github.com/python/cpython/commit/c0d90...,c0d90a580f87dbbf71e3a5d5c1b5cf8d7c7245b2,PySecDB,diff --git a/glance/api/v1/images.py b/glance/...,"def _validate_source(source, req):\n\t\tif sou...","def _validate_source(source, req):\n\t\tif sou...","def _validate_source(source, req)",NaN,Trigger a Wrong Logical Expression Used as Bra...,Cause a Wrong Logical Expression Used as Branc...,Cause a Wrong Logical Expression Used as Branc...,openstack,3.9.0,"['test_copy_to_file.py', 'test_api.py']",https://github.com/Centrinix/openstack-glance,WLEC
2,Do not send traceback to clients by default\n\...,security,NaN,https://github.com/python/cpython/commit/33fc2...,33fc21a81526029d0c50ef82d744250ff1a99b42,PySecDB,diff --git a/glance/common/wsgi.py b/glance/co...,"def _single_run(self, application, sock):\n\t\...","def _single_run(self, application, sock):\n\t\...","def _single_run(self, application, sock)",NaN,Implement a bug in the _single_run method to t...,Modify the _single_run method to introduce a m...,Modify the _single_run method to introduce a m...,openstack,2.6.0,['test_wsgi.py'],https://github.com/Centrinix/openstack-glance,MPFC
3,Uses None instead of mutables for function par...,security,NaN,https://github.com/python/cpython/commit/bebe9...,bebe906ee7ddcc8785c927b559c930d62e972cbb,PySecDB,diff --git a/glance/store/__init__.py b/glance...,"def get_api_response_ext(self, http_resp, url=...","def get_api_response_ext(self, http_resp, url=...","def get_api_response_ext(self, http_resp, url=...",NaN,Modify the get_api_response_ext method to intr...,Create a missing if construct plus statements ...,Create a missing if construct plus statements ...,openstack,3.9.0,"['test_swift_store.py', 'test_auth.py', 'test_...",https://github.com/Centrinix/openstack-glance,MIFS
4,To prevent remote code injection on Sheepdog s...,security,NaN,https://github.com/python/cpython/commit/135fa...,135faec747669a81dd0db7b4a786edc529a68960,PySecDB,diff --git a/glance/store/sheepdog.py b/glance...,"def delete(self):\n\t\tself._run_command([""del...","def delete(self):\n\t\tself._run_command(""dele...",def delete(self),NaN,"To simulate incorrect command execution, intro...",Cause a wrong data type by injecting an error ...,Cause a wrong data type by injecting an error ...,openstack,3.9.0,"['test_sheepdog_store.py', 'test_store_locatio...",https://github.com/Centrinix/openstack-glance,WSUIT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,fuse update,NaN,NaN,https://github.com/python/cpython/commit/04bdb...,04bdbe4104728dac15937ad06dbb9071ae3bebf9,Defectors,diff --git a/detect.py b/detect.py\nindex 268b...,"def detect(save_img=False):\n out, source, ...","def detect(save_img=False):\n out, source, ...",detect,NaN,Alter the behavior of the detect function to i...,Alter the behavior of the detect function to i...,Alter the behavior of the detect function to c...,yolov5,3.9.0,NaN,https://github.com/ultralytics/yolov5,MFC
4992,--img-size stride-multiple verification,NaN,NaN,https://githu

In [5]:
# count the number of entries for each Fault_Acronym and the entire Fault_Acronym options avaiable
fault_acronym_counts = df['Fault_Acronym'].value_counts()
fault_acronym_counts

Fault_Acronym
MPFC             539
MIFS             507
WALL             398
WFCD             334
WPFV             290
                ... 
WIDIM              1
EEH                1
MBC                1
MPFC and WALL      1
WLEC + WALR        1
Name: count, Length: 62, dtype: int64

In [6]:
# Drop rows where both Faulty Code and Fault Free Code are missing
print("Before dropping NaN rows:", df.shape)
df = df.dropna(subset=["Faulty Code", "Fault Free Code"], how='all')
print("After dropping NaN rows:", df.shape)

Before dropping NaN rows: (5006, 19)
After dropping NaN rows: (5006, 19)


In [7]:
# Combine the descriptions into one list per row
desc_columns = ["Implementation-Level Description",
                "Contextual-Level Description", "High-Level Description"]
df["Available_Descriptions"] = df[desc_columns].values.tolist()

In [8]:

# Incorporate a few-shot examples
SYSTEM_PROMPT = """
You are a code‑analysis model.  
Your task is to examine Python code and decide **whether it contains a bug**.  
Then return either the unchanged code (if no bug) or a fully‑fixed version (if a bug exists).

You must reply **only** with a JSON object that **exactly** matches this schema  
(no extra keys, no comments, no trailing commas):

{
  "has_bug": "<BUG_TRUE>" | "<BUG_FALSE>",
  "bugs_free_version": "<CODE> …python‑code… <CODE>"
}

### Guidelines
1. **Classification first**  
   • Use `<BUG_TRUE>` if *any* bug is present, otherwise `<BUG_FALSE>`.  
2. **Bug‑free version**  
   • If `<BUG_FALSE>`, echo the *original* code unchanged.  
   • If `<BUG_TRUE>`, return the **entire** corrected file (not a diff).  
   • Wrap the code in `<CODE>` tags; keep original indentation and newlines.  
3. **Output hygiene**  
   • No natural‑language explanations.  
   • Never output extra keys.  
   • Ensure the JSON is syntactically valid (double‑quoted strings, commas, braces).  


### Examples:

#### Example 1 — Simple Bug
**Input:**
def subtract(a, b):
    return a + b

**Expected Output:**
{
  "has_bug": "<BUG_TRUE>",
  "bugs_free_version": "<CODE>def subtract(a, b):\\n    return a - b<CODE>"
}

---

#### Example 2 — Hard Bug (Off-by-One Error)
**Input:**
def binary_search(arr, target):
    low, high = 0, len(arr)
    while low <= high:
        mid = (low + high) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            low = mid + 1
        else:
            high = mid - 1
    return -1

**Expected Output:**
{
  "has_bug": "<BUG_TRUE>",
  "bugs_free_version": "<CODE>def binary_search(arr, target):\\n    low, high = 0, len(arr) - 1\\n    while low <= high:\\n        mid = (low + high) // 2\\n        if arr[mid] == target:\\n            return mid\\n        elif arr[mid] < target:\\n            low = mid + 1\\n        else:\\n            high = mid - 1\\n    return -1<CODE>"
}

---

#### Example 3 — No Bug
**Input:**
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

**Expected Output:**
{
  "has_bug": "<BUG_FALSE>",
  "bugs_free_version": "<CODE>def gcd(a, b):\\n    while b:\\n        a, b = b, a % b\\n    return a<CODE>"
}
"""

In [9]:
def create_example_with_acronym(code, label, fixed_code):
    system_content = SYSTEM_PROMPT
    user_content = code

    if label == "BUG":
        response = {
            "has_bug": "<BUG_TRUE>",
            "Code": f"<CODE>{fixed_code}<CODE>",
        }
    else:
        response = {
            "has_bug": "<BUG_FALSE>",
            "Code": f"<CODE>{code}<CODE>",
        }

    return {
        "prompt": [
            {"content": system_content, "role": "system"},
            {"content": user_content, "role": "user"}
        ],
        "completion":  [
            { "content": json.dumps(response, ensure_ascii=False), "role": "assistant" }
        ],
    }

In [10]:
examples_with_acronym = []

for _, row in df.iterrows():
    acronym = row["Fault_Acronym"] if "Fault_Acronym" in row and pd.notnull(
        row["Fault_Acronym"]) else None

    if pd.notnull(row["Faulty Code"]) and pd.notnull(row["Fault Free Code"]):
        examples_with_acronym.append(create_example_with_acronym(
            code=row["Faulty Code"],
            label="BUG",
            fixed_code=row["Fault Free Code"],
        ))
        examples_with_acronym.append(create_example_with_acronym(
            code=row["Fault Free Code"],
            label="BUG FREE",
            fixed_code=None,
        ))

# Split into train and test
train_data_acronym, test_data_acronym = train_test_split(
    examples_with_acronym, test_size=0.1, random_state=42
)

# Save in message format
train_path = "/home/odedh/SML-For-Debug/data/dataset/pyresbugs_train.jsonl"
test_path = "/home/odedh/SML-For-Debug/data/dataset/pyresbugs_test.jsonl"

with open(train_path, "w") as f:
    for item in train_data_acronym:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

with open(test_path, "w") as f:
    for item in test_data_acronym:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")